In [10]:
# .env 파일에서 환경 변수를 불러오기 위한 라이브러리
from dotenv import load_dotenv
import os

# .env 파일의 경로를 지정하여 환경 변수를 로드합니다.
load_dotenv(dotenv_path='../.env')

# 환경 변수에서 OpenAI API 키를 가져옵니다.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


### 문제 1-1: 기본 Chain 만들기 - AI 요리사

In [14]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


# ChatOpenAI 모델을 초기화합니다.
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="openai/gpt-oss-120b",
    temperature=0.7
)

# 프롬프트 템플릿을 정의합니다. 
# 이 템플릿은 사용자로부터 재료를 입력받아 AI에게 요리 추천을 요청하는 역할을 합니다.
# 프롬프트 템플릿을 수정하여 예상 실행 결과와 동일한 형식을 갖도록 합니다.
template_text = """
당신은 요리사입니다. 사용자가 제공하는 재료를 사용하여 만들 수 있는 요리법을 아래 형식에 맞춰 추천해주세요.

---
[입력 재료]: {ingredients}
---
[출력 형식]:{ingredients}로 만들 수 있는 요리를 추천드립니다!

추천 요리: [요리 이름]
재료: {ingredients}
조리법:
"""

# PromptTemplate.from_template() 함수를 사용하여 프롬프트 템플릿 객체를 생성합니다.
prompt_template = PromptTemplate.from_template(template_text)

# StrOutputParser를 사용하여 모델의 출력 결과를 문자열로 변환합니다.
output_parser = StrOutputParser()

# LCEL(LangChain Expression Language)을 사용하여 프롬프트, 모델, 출력 파서를 연결하는 체인을 생성합니다.
chain = prompt_template | llm | output_parser

# 1. 재료를 변수에 저장합니다.
ingredients = "계란, 밥, 김치"

# 체인을 실행하고 결과를 출력합니다.
# 'ingredients' 변수에 "계란, 밥, 김치"를 전달하여 체인을 호출합니다.
response = chain.invoke({"ingredients": ingredients})

# 최종 결과를 출력합니다.
print(f'입력: "{ingredients}"')
print("출력:")
print(response)


입력: "계란, 밥, 김치"
출력:
계란, 밥, 김치로 만들 수 있는 요리를 추천드립니다!

추천 요리: 김치볶음밥 (계란 토핑)
재료: 계란, 밥, 김치
조리법:
1. **재료 준비**  
   - 김치를 작게 썰어 둡니다.  
   - 밥은 찬밥이 좋으며, 전날 남은 밥을 사용하면 더욱 고슬고슬합니다.  
   - 계란은 반숙 혹은 반숙 위에 올릴 경우 노른자를 살짝 풀어 둡니다.

2. **김치 볶기**  
   - 팬에 식용유(또는 참기름) 1큰술을 두르고 중불에서 달군다.  
   - 다진 마늘 1/2작은술을 넣고 향이 올라올 때까지 볶는다.  
   - 썰어 둔 김치를 넣고 3~4분간 중간 불에서 볶아 김치가 부드러워지고 약간 물기가 날아갈 때까지 볶는다.  

3. **밥 넣기**  
   - 김치가 충분히 볶아졌으면 밥을 한 공기 넣고 고루 섞는다.  
   - 간장 1작은술, 고춧가루 1작은술, 설탕 ½작은술, 참기름 1작은술을 넣고 다시 한 번 골고루 볶아준다.  
   - 필요에 따라 소금·후추로 간을 맞춘다.  

4. **계란 프라이**  
   - 다른 팬에 식용유를 살짝 두르고 중불에서 달군 뒤, 계란을 깨뜨려 넣는다.  
   - 흰자가 반쯤 익을 때까지 기다린 뒤, 노른자를 살짝 터뜨려 반숙 상태로 마무리한다. (노른자를 완전히 익히고 싶다면 완전히 익혀도 좋다.)

5. **플레이팅**  
   - 볶은 김치밥을 그릇에 담고, 그 위에 프라이한 계란을 올린다.  
   - 원한다면 김가루, 파썰기, 김치 조각을 고명으로 올려 색감과 풍미를 더한다.  

6. **완성**  
   - 따뜻한 김치볶음밥 위에 부드러운 계란 노른자가 흐르며, 매콤한 김치와 고소한 밥, 부드러운 계란이 어우러진 한 끼가 완성됩니다. 즐겁게 드세요!


### 문제 1-2 : 2단계 Multi Chain 만들기 - 여행지 정보 시스템

In [15]:
# 필요한 라이브러리들을 가져옵니다.
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

# 1단계 체인: 여행지의 대표 명소 추천
prompt1 = ChatPromptTemplate.from_messages([
    ("system", "당신은 특정 도시나 국가의 대표적인 관광 명소 1가지를 추천하는 여행 전문가입니다."),
    ("user", "{topic}의 대표적인 관광 명소 1가지를 추천해주세요. 명소의 이름만 간결하게 답변해주세요.")
])

chain1 = prompt1 | llm | StrOutputParser()


# 2단계 체인: 명소의 상세 정보 제공
prompt2 = ChatPromptTemplate.from_messages([
    ("system", "당신은 특정 관광 명소에 대한 상세 정보를 제공하는 역사학자이자 여행 가이드입니다."),
    ("user", "{landmark}에 대한 상세한 정보를 역사, 특징, 방문 팁을 포함하여 설명해주세요.")
])

chain2 = prompt2 | llm | StrOutputParser()


# 두 체인을 연결하여 전체 체인 생성
# RunnableParallel을 사용하여 각 단계를 동시에 실행하고 결과를 함께 볼 수 있도록 구성합니다.
# 1. 'landmark' 키에는 chain1의 결과(명소 이름)를 할당합니다.
# 2. 'details' 키에는 chain1의 결과를 받아 chain2로 전달하여 상세 정보를 가져옵니다.
final_chain = RunnableParallel(
    landmark=chain1,
    details=chain1 | chain2
)

# 체인 실행
input_city = "프랑스"
result = final_chain.invoke({"topic": input_city})

# 결과 출력
print(f'입력: "{input_city}"')
print("-" * 30)
print(f'1단계 결과: "{result["landmark"]}"')
print("-" * 30)
print(f'2단계 결과:\n명소: {result["landmark"]}\n{result["details"]}')

입력: "프랑스"
------------------------------
1단계 결과: "에펠탑"
------------------------------
2단계 결과:
명소: 에펠탑
## 에펠탑 (Eiffel Tower) – 역사·특징·방문 팁 종합 안내  

---

### Ⅰ. 역사  

| 연도 | 주요 사건 | 의미·배경 |
|------|-----------|-----------|
| **1884년 1월** | 구스타프 에펠(​Gustave Eiffel)과 그의 엔지니어 팀이 파리 시청으로부터 **제2차 세계 박람회(Exposition Universelle, 1889)** 를 위한 설계 의뢰를 받음 | 파리 100주년을 기념하고, 당시 가장 높은 인공 구조물(300 m)을 만들겠다는 야심찬 목표가 제시됨 |
| **1887년 1월** | 기초 공사 시작 (강철 기둥 18,038개, 무게 7,300 톤) | 파리 시민들의 반대와 “철의 괴물”이라는 비난에도 불구하고, 2년 2개월 만에 완공을 목표로 착공 |
| **1889년 3월 31일** | **완공** 및 **시연 행사** (전등 점등, 엘리베이터 시연) | 41 일간 전시 기간 동안 2,000만 명 이상의 관람객이 방문, 당시 세계에서 가장 높은 구조물(324 m)로 기록 |
| **1900년** | 제2차 파리 박람회에 재사용, 전기 조명 시스템 개선 | 전기 조명(2,000 개)으로 야경을 장식, ‘빛의 탑’이라는 별명이 붙음 |
| **1910년** | **에펠탑 전용 전력망** 구축 (자체 발전소) | 전기 공급의 독립성을 확보해 파리 전력망에 큰 영향을 미침 |
| **1937년** | 프랑스-독일 전쟁 전후, **전쟁 방어용 안테나** 설치 | 통신·라디오 전파 실험을 위한 전략적 활용 |
| **1940~1944년** | **독일 점령** 시점에 전시관과 엘리베이터가 파손·폐쇄 | 전쟁 후 복구 작업을 거쳐 1946년 재개장 |
| **1985년** | **에펠탑 100주년** 기념 

### 문제 1-3 : FewShotPromptTemplate과 시스템 메시지 활용 

In [16]:
# 필요한 라이브러리들을 가져옵니다.
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser

# 1. Few-Shot 학습을 위한 예시 데이터 정의
examples = [
    {
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
        "keywords": "세계보건기구, 건강 위기, 국제 협력"
    },
    {
        "news": "영화 '범죄도시4'가 개봉 10일 만에 천만 관객을 돌파하며 흥행 신기록을 세웠다. 주연 배우 마동석의 시원한 액션 연기가 관객들의 큰 호응을 얻고 있다.",
        "keywords": "범죄도시4, 천만 관객, 마동석"
    }
]

# 2. 각 예시의 형식을 정의하는 프롬프트 템플릿 생성
# 사용자의 뉴스 기사({news})와 AI의 답변({keywords}) 형식을 지정합니다.
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])

# 3. Few-Shot 프롬프트 템플릿 생성
# 위에서 정의한 예시 데이터와 형식 템플릿을 결합합니다.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 4. 최종 프롬프트 템플릿 생성
# 시스템 메시지, Few-Shot 예시, 그리고 실제 사용자 입력을 모두 포함하는 최종 프롬프트를 만듭니다.
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 뉴스 기사의 핵심 키워드 3개를 정확하게 추출하는 AI 전문가입니다. '키워드: ' 형식으로 답변해주세요."),
    few_shot_prompt,  # AI에게 학습시킬 예시들을 여기에 삽입합니다.
    ("human", "{input}")   # 실제 분석을 요청할 뉴스 기사가 들어갈 자리입니다.
])


# 5. 체인 생성 및 실행
chain = final_prompt | llm | StrOutputParser()

test_news = "제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."

result = chain.invoke({"input": test_news})

# 결과 출력
print(f"입력 뉴스:\n{test_news}")
print("-" * 30)
print(f"추출 결과:\n{result}")

입력 뉴스:
제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다.
------------------------------
추출 결과:
키워드: 제미나이 2.0, 구글 AI 스튜디오, 멀티모달 입력/출력
